In [ ]:
# case study 
# exp baseline artistic-toad  : powerful-crow type 2    6个比赛4个比基线高 curious-cattle type 1    6个比赛5个比基线高 
import pandas as pd 
import pickle


In [ ]:
baseline = "artistic-toad"

type_1 = "humble-cardinal"#"powerful-crow"

type_2 = "curious-cattle"


In [ ]:
comp_list = [
    "us-patent-phrase-to-phrase-matching",
    "tweet-sentiment-extraction",
    "jigsaw-toxic-comment-classification-challenge", "cassava-leaf-disease-classification","leaf-classification","whale-categorization-playground",
    "h-and-m-personalized-fashion-recommendations"]

In [ ]:
EXP = [type_1,type_2]

In [ ]:
F = []

for exp in EXP:
    p0= f'/data/share_folder_local/amlt/{exp}/combined_logs/summary.pkl'
    with open(p0, 'rb') as f:
        summary  = pickle.load(f)
    for idx, competition in enumerate(summary.keys()):
        if competition.split(".")[-2] in EVO:
            data = summary[competition]
            print(f"Experiment: {exp}, Competition: {competition}")
            test_scores_df = pd.DataFrame.from_dict(data["test_scores"], orient="index", columns=["Test Score"])
            test_scores_df.index.name = "Loop"
            
            # valid_scores_dict = data["valid_scores"]
            # # 提取 ensemble 验证分数
            # ensemble_scores = {}
            # for loop_id, df in valid_scores_dict.items():
            #     if "ensemble" in df.index:
            #         ensemble_scores[loop_id] = df.loc["ensemble"].iloc[0]
            ensemble_scores = test_scores_df["Test Score"]
            #print(ensemble_scores)
            bronze_threshold = data["bronze_threshold"]
            silver_threshold = data["silver_threshold"]
            if ensemble_scores.size > 0:
                if bronze_threshold > silver_threshold:
                    F.append((exp, competition, ensemble_scores.min(), -1))
                else:
                    F.append((exp, competition, ensemble_scores.max(), 1))

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
rows = []
for comp, score_dict in comp2scores.items():
    scores = list(score_dict.values())
    if not scores:
        continue
    rows.append({
        "competition": comp,
        "mean": np.mean(scores),
        "std": np.std(scores),
        "n": len(scores),
    })

df = pd.DataFrame(rows).sort_values("mean", ascending=False)
print(df)

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np

data = F  # F 内部为 (exp, comp, score, direction)

def strip_version(name):
    if "." in name:
        return name.rsplit(".", 1)[0]
    return name

# 第一步：按 competition + experiment 聚合所有 score
comp2scores_raw = defaultdict(lambda: defaultdict(list))
comp_direction = {}  # 每个 competition 的方向，只需要记录一次

for exp, comp, score, direction in data:
    comp_base = strip_version(comp)
    comp2scores_raw[comp_base][exp].append(score)
    comp_direction[comp_base] = direction

# 第二步：求平均值
comp2scores = defaultdict(dict)
for comp, exp_dict in comp2scores_raw.items():
    direction = comp_direction[comp]   # 1 or -1
    for exp, score_list in exp_dict.items():
        comp2scores[comp][exp] = (np.mean(score_list),np.std(score_list))
        # if direction == 1:
        #     comp2scores[comp][exp] = np.mean(score_list)
        # else:
        #     comp2scores[comp][exp] = np.mean(score_list)

# target = "curious-cattle"

# for comp, score_dict in comp2scores.items():

#     if target not in score_dict:
#         continue

#     direction = comp_direction[comp]  #
#     base = score_dict[target]

    # diffs = {exp: (score - base) * direction for exp, score in score_dict.items()}

    # sorted_exps = sorted(diffs, key=diffs.get, reverse=True)
    # sorted_vals = [diffs[e] for e in sorted_exps]

    # plt.figure(figsize=(5, 4))
    # plt.barh(sorted_exps, sorted_vals)
    # plt.axvline(0, color='black', linewidth=1)
    # plt.title(f"Difference from humble-cardinal — {comp}")
    # plt.xlabel("Performance Difference (positive = better)")
    # plt.tight_layout()
    # plt.show()
    # plt.savefig(f'diff_{comp}.png')


In [ ]:
import pandas as pd

rows = []
for comp, exp_dict in comp2scores.items():
    for exp, (mean, std) in exp_dict.items():
        rows.append({
            "competition": comp,
            "experiment": exp,
            "mean": mean,
            "std": std,
        })

df = pd.DataFrame(rows)
print(df.head())


In [ ]:
name_map = {
    "artistic-toad": "RD-Agent-GPT5",
    "humble-cardinal": "RD-Agent-GPT5-world-model-type2",
    "curious-cattle": "RD-Agent-GPT5-world-model-type1",
}
df["experiment"] = df["experiment"].map(name_map)
df.to_csv("competition_experiment_stats_renamed.csv", index=False)


In [ ]:
special_exp= ["artistic-toad"] +["humble-cardinal"] +["curious-cattle"]

In [ ]:
df